# Imputación con interpolacion

## Preparar librerias

In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import random
import warnings
#import math
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#from sklearn.preprocessing import Normalizer
import importlib.util

# Especifica la ruta al archivo del módulo
ruta_al_modulo = './util/auxi.py'

# Importa el módulo
spec = importlib.util.spec_from_file_location("auxi", ruta_al_modulo)
auxi = importlib.util.module_from_spec(spec)
spec.loader.exec_module(auxi)

## Leer datos originales

Hay que leer el dataset original para comparar los datos imputados con los reales

Temperatura

In [2]:
warnings.filterwarnings('ignore', category=SyntaxWarning)  # Porque el \d de \data y \daily lo toma como un valor especial
dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)
serie_temperatura = dataframe_temperatura.squeeze('columns')
serie_temperatura = serie_temperatura.resample('D').mean()
serie_temperatura['1984-12-31'] = (serie_temperatura['1984-12-30'] + serie_temperatura['1985-01-01'])/2
serie_temperatura['1988-12-31'] = (serie_temperatura['1988-12-30'] + serie_temperatura['1989-01-01'])/2
serie_temperatura.head()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\alaba\AppData\Local\Temp\ipykernel_22480\790941083.py:2: SyntaxWarning: invalid escape sequence '\d'
  dataframe_temperatura = pd.read_csv("..\data\daily-minimum-temperatures-in-me.csv", header=0, index_col=0, parse_dates=True)


Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Freq: D, Name: Daily minimum temperatures in Melbourne, Australia, 1981-1990, dtype: float64

Cardiovascular

In [3]:
dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)
serie_cardiovascular = dataframe_cardiovascular.squeeze('columns')
serie_cardiovascular = serie_cardiovascular.astype(float)
serie_cardiovascular.head()

C:\Users\alaba\AppData\Local\Temp\ipykernel_22480\3747356855.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataframe_cardiovascular = pd.read_csv("..\data\\ads_hour.csv", header=0, index_col=0, parse_dates=True)


Date
2017-08-03 00:00:00    49136.0
2017-08-03 01:00:00    46450.0
2017-08-03 02:00:00    46355.0
2017-08-03 03:00:00    43748.0
2017-08-03 04:00:00    42281.0
Name: ads, dtype: float64

Manchas solares

In [4]:
dataframe_sunspots = pd.read_csv("..\data\monthly-sunspots.csv", header=0, index_col=0, parse_dates=True)
serie_sunspots = dataframe_sunspots.squeeze('columns')
serie_sunspots = serie_sunspots.resample('ME').mean() # Para los meses
serie_sunspots.head()

Month
1749-01-31    58.0
1749-02-28    62.6
1749-03-31    70.0
1749-04-30    55.7
1749-05-31    85.0
Freq: ME, Name: Sunspots, dtype: float64

## Interpolacion

In [5]:
def imputar_interpolacion(serie, lista):
    dataframe_inter = pd.DataFrame({
        'Dataset': [],
        'RMSE': [],
        'MAE': [],
        'MAPE': [],
        'BIAS': []
    })
    for serie_nulo in lista:

        serie_inter = serie_nulo.copy()
        # Tratar problema de nulos al inicio, interpolate no los puede tratar bien
        parada=False
        j=0
        while(j<len(serie_inter) and parada!=True):
            valor = serie_inter.iloc[j]
            if np.isnan(valor):
                serie_inter.iloc[j] = serie_inter.mean()
            else:
                parada=True
            j+=1

        # Tratar problema de nulos al final, interpolate no los puede tratar bien
        parada=False
        j=len(serie_inter)-1
        while(j>=0 and parada!=True):
            valor = serie_inter.iloc[j]
            if np.isnan(valor):
                serie_inter.iloc[j] = serie_inter.mean()
            else:
                parada=True
            j-=1
        
        # Imputamos con interpolacion
        serie_inter = serie_inter.interpolate(method='linear')
        #print(serie_inter.head(3))

        # Calculamos la raiz del error cuadrático medio con las series imputadas con interpolacion
        rmse_inter = auxi.RMSE(serie, serie_nulo, serie_inter)
        # Calculamos el error medio absoluto con las series imputadas con interpolacion
        mae_inter = auxi.MAE(serie, serie_nulo, serie_inter)
        # Calculamos el error medio absoluto porcentual con las series imputadas con interpolacion
        mape_inter = auxi.MAPE(serie, serie_nulo, serie_inter)
        # Calculamos el bias con las series imputadas con interpolacion
        bias_inter = auxi.BIAS(serie, serie_nulo, serie_inter)

        nueva_fila = pd.DataFrame({ 'Dataset': serie_nulo.name, 'RMSE': [rmse_inter], 'MAE': [mae_inter], 'MAPE': [mape_inter], 'BIAS': [bias_inter]})
        dataframe_inter = pd.concat([dataframe_inter, nueva_fila], ignore_index=True)
    return dataframe_inter

## Listar

In [6]:
lista_series_temperatura = auxi.leer_series('..\miss_valued_data_temperatura\\')
lista_series_cardiovascular = auxi.leer_series('..\miss_valued_data_cardiovascular\\')
lista_series_sunspots = auxi.leer_series('..\miss_valued_data_manchas_solares\\')

## Escalar datos

### Temperatura

In [7]:
min_max_scaler_temperatura = MinMaxScaler().fit(serie_temperatura.values.reshape(-1,1))
serie_original_temperatura_escalada = auxi.aplicar_escalado(serie_temperatura, min_max_scaler_temperatura)
series_temperatura_escaladas = auxi.escalar(min_max_scaler_temperatura, lista_series_temperatura)

### Cardiovascular

In [8]:
min_max_scaler_cardiovascular = MinMaxScaler().fit(serie_cardiovascular.values.reshape(-1,1))
serie_original_cardiovascular_escalada = auxi.aplicar_escalado(serie_cardiovascular, min_max_scaler_cardiovascular)
series_cardiovascular_escaladas = auxi.escalar(min_max_scaler_cardiovascular, lista_series_cardiovascular)

### Manchas solares

In [9]:
min_max_scaler_sunspots = MinMaxScaler().fit(serie_sunspots.values.reshape(-1,1))
serie_original_sunspots_escalada = auxi.aplicar_escalado(serie_sunspots, min_max_scaler_sunspots)
series_sunspots_escaladas = auxi.escalar(min_max_scaler_sunspots, lista_series_sunspots)

## Imputamos interpolación

In [10]:
dataframe_temperatura_inter = imputar_interpolacion(serie_temperatura, lista_series_temperatura)
dataframe_cardiovascular_inter = imputar_interpolacion(serie_cardiovascular,  lista_series_cardiovascular)
dataframe_manchas_solares_inter = imputar_interpolacion(serie_sunspots, lista_series_sunspots)
dataframe_general_inter = pd.concat([dataframe_temperatura_inter, dataframe_cardiovascular_inter, dataframe_manchas_solares_inter], ignore_index=True)
dataframe_general_inter

,Dataset,RMSE,MAE,MAPE,BIAS
0,min_temperature_bateria_10.csv,1.032988,0.256995,2.817399,-0.004436
1,min_temperature_bateria_15.csv,1.369825,0.425133,4.260352,-0.135515
2,min_temperature_bateria_20.csv,1.656718,0.591620,6.090818,-0.198220
3,min_temperature_error_medida_10.csv,0.693817,0.170934,1.797728,0.011980
4,min_temperature_error_medida_15.csv,0.845862,0.251451,2.630531,0.014389
5,min_temperature_error_medida_20.csv,0.999040,0.341294,3.496302,0.020872
6,min_temperature_interferencias_10.csv,0.746303,0.182032,1.791032,-0.012185
7,min_temperature_interferencias_15.csv,0.929418,0.280230,2.767899,-0.023768
8,min_temperature_interferencias_20.csv,1.134056,0.389091,3.894535,0.023918
9,min_temperature_mix_10.csv,0.842076,0.198999,1.931381,-0.027683
